# Berichtsheft – nanoServices Architektur & Stand (2024-08-02)

**Projekt:** Entwicklung eines semantisch integrierten Microservices-Frameworks (nanoServices)

---

## Projektüberblick

Das Ziel des nanoServices-Frameworks ist die Entwicklung einer semantisch integrierten Microservices-Plattform auf Basis von Apache Jena Fuseki (für RDF-Daten), Temporal (für Workflow-Orchestrierung), Supervisor (für Prozessmanagement), Docker (für Containerisierung) und weiteren Kernkomponenten. Das System zeichnet sich durch vollständige lose Kopplung, deklarative Selbstbeschreibung der Services sowie ein konsequentes Monitoring- und Logging-Konzept aus.

---

## Kernkomponenten

- **Fuseki:** Zentrale RDF-Datenhaltung, alle Service- und System-Metadaten werden hier im „Nanoversum“ abgelegt.
- **Supervisor:** Prozessüberwachung, Steuerung und Logmanagement aller Worker-Prozesse.
- **Config-Worker:** Nimmt Service-Registrierungen entgegen, verwaltet Policies (z. B. Logrotation), ist Ankerpunkt für System- und Monitoring-Workflows (inkl. Logrotation als Temporal-Task).
- **Temporal:** Orchestriert alle System- und Business-Workflows, verwaltet History, Retries, Heartbeats etc.
- **nanoservices-Frontend:** Zentrales Dashboard für Monitoring, Visualisierung und Steuerung.
- **Docker:** Containerisierung sämtlicher Dienste, unterstützt durch einen eigenen „docker-worker“ zur Orchestrierung und Statusüberwachung.
- **LodView/Lod4All:** Zusatztools für die Visualisierung und Abfrage von RDF-Daten, insbesondere in Entwicklungsumgebungen.

---

## Datenflüsse und Verantwortlichkeiten


| Komponente     | Aufgabe                                                   | Hauptspeicher   |
|----------------|-----------------------------------------------------------|-----------------|
| Supervisor     | Start/Stop/Monitoring, Logging von Prozessen              | Logfiles        |
| Fuseki         | Alle Self-Descriptions, Snapshots, Policies, Relationen   | RDF             |
| Config-Worker  | Registrierung, Heartbeat, Policy Mgmt, Logrotation        | RDF/Temporal    |
| Temporal       | History, Scheduling, Retries, Workflows, Events           | Temporal DB     |
| Docker-Worker  | Container-Status, Logs, Orchestrierung                    | Docker-API      |
| Frontend       | Monitoring, Visualisierung, Steuerung                     | (über API)      |

---

## Logging und Logrotation

Das System setzt auf strukturierte Logs im **JSON Lines**-Format, sowohl für Worker als auch für Supervisor-Prozesse.  
Die Logrotation erfolgt zentral als **Temporal-Task** auf dem Config-Worker und wird gemäß definierter Policies (z. B. max Days/Size) durchgeführt.  
Alle relevanten Logrotationen und -ereignisse werden als Events im RDF-Modell (Fuseki) dokumentiert.

---

## Service-Registrierung und Self-Description

Jeder neue Worker-Service meldet sich beim System zentral über einen Temporal-Workflow (`registerService`) mit seiner eigenen OWL-S/RDF-Selbstauskunft, Prozessstatus (Supervisor), sowie temporalen Daten (TaskQueues, Status) an.  
Alle Informationen werden in Fuseki als Snapshot abgelegt, und optional werden regelmäßige Heartbeats für die Statusüberwachung verwendet.

---

## Docker-Integration

Ein dedizierter **docker-worker** macht sämtliche Container-, Log- und Statusdaten systemweit über eine REST-API zugänglich.  
Auch die Logfiles der Container werden zentral und strukturiert bereitgestellt.  
Volumes, Netzwerke und Images können ebenfalls im RDF-Modell abgebildet werden.

---

## Frontend und Monitoring

Das Main-Frontend bietet:
- Live-Übersicht aller registrierten Worker und Services
- Visualisierung von Logs, Heartbeats, Status und Fehlern
- Service-Discovery und Anzeige der Self-Descriptions, Workflows/Activities
- Integration mit Docker- und Supervisor-Statusdaten
- Unterstützung von Dev-/Prod-Features (z. B. LODView für RDF-Browsing)

---

## Policies & RDF-Modell

Systemweite Policies (z. B. für Logrotation, Heartbeat, Restart, Retention) werden als RDF-Vokabular beschrieben und versioniert im Fuseki-System abgelegt.  
Dies ermöglicht deklarative Systemkonfiguration und transparente Auditierbarkeit aller Systemereignisse und Wartungsprozesse.

---

## Offene ToDos / Nächste Schritte (Stand: 20240802)

- Implementierung und Standardisierung der Worker-Registrierung per Temporal-Workflow
- Vollständige Logrotation als Temporal-Task und Events im RDF
- Ausbau des docker-worker zur systemweiten Überwachung und Steuerung aller Container
- Weiterentwicklung des Main-Frontends mit Schwerpunkt auf Logs, Policies und Service-Discovery
- Definition und Erweiterung des nano:-RDF-Vokabulars für Worker, Policies, Events und Systemrollen
- Modellierung und erste Umsetzung eines User-/Rollen- und Berechtigungssystems (später, optional mit Solid)

---

## Ausblick

Mit dieser Architektur wird ein vollständig lose gekoppeltes, robustes und deklarativ beschreibbares Microservice-Ökosystem geschaffen, das höchste Transparenz, Wartbarkeit und Erweiterbarkeit gewährleistet.  
Alle wesentlichen Vorgänge und Komponenten sind nachvollziehbar dokumentiert, was auch langfristig die Weiterentwicklung und Integration neuer Services stark vereinfacht.

---